In [26]:
import pandas as pd
from copy import deepcopy
import plotly.graph_objects as go

def detect_frequency(pd_data:pd.DataFrame, window_size = 5):
    pd_data = pd_data.sort_values(by=['time_key'])

    window_start = pd_data.iloc[0]['time_key']
    window_end = window_start + pd.Timedelta(seconds=window_size)
    window_inner_number = 0

    single_window_statisc = {
        'start_time': '',
        'end_time' : '',
        'count': ''
    }
    results = []

    i = 0
    leng = len(pd_data)
    while (i < leng):
        d = pd_data.iloc[i]
        if d['time_key'] >= window_start and d['time_key'] < window_end:
            window_inner_number += 1
            i += 1
        else:
            statis = deepcopy(single_window_statisc)
            statis['start_time'] = window_start
            statis['end_time'] = window_end
            statis['count'] = window_inner_number
            results.append(statis)

            window_start = window_end
            window_end = window_start + pd.Timedelta(seconds=window_size)
            window_inner_number = 0
    
    return results

def plot(data, channel_name, window_size = 5):
    marker_size = 1
    line_width = 1.5

    xs = [t['start_time'] for t in data]
    ys = [y['count'] for y in data]

    fig = go.Figure() 
    fig.add_trace(
        go.Scattergl(
            mode = 'markers',
            marker = dict(
                size = marker_size
            ),
            line={'width': line_width},
            x=xs, 
            y=ys, 
            name=channel_name,
            yaxis='y'),
    )


    fig.update_xaxes(tickfont=dict(
                                    size=20,
                                    ) )

    fig.update_yaxes(tickfont=dict(
                                    size=20,
                                    ) )
    fig.update_layout(
        title = dict(
            text = fr'Frequency of {channel_name} that TransientHydraulics algorithm really received in every {window_size} seconds',
            font = dict(size = 30)
        ),
        yaxis=dict(title=dict(
            text = "count",
            font = dict(size = 20)
            ), 
            )
            ,
        xaxis=dict(title=dict(
            text = "Time",
            font = dict(size = 20)
            )),
        legend = dict(font = dict(size = 20))
            )

    # fig.write_html(rf"C:\Users\Jyu20\Downloads\tnd_simulation_validation_{well_name}.html")
    # fig.write_html('test_ecd.html')
    return fig
    # fig.show()




In [17]:
data = pd.read_csv(r"C:\NotOneDrive\Data\AVF_download\Fix_HCI_reset_bug_2236031\DRILLING_VFLOW.csv", skiprows=[1]).rename(columns={'time': 'time_key', 'DRILLING_VFLOW': 'value'})
data['time_key'] = pd.to_datetime(data['time_key'])
frequency = detect_frequency(data)

In [ ]:


fig = plot(frequency, 'flowrate', 5)
fig.show()